# 信号处理仿真与应用-MWorks版-案例

# 第5章 变换、相关性和建模

## 5.1 变换

### 5.1.2 离散傅里叶变换

离散傅里叶变换（即DFT）是数字信号处理的首要工具。该产品的基础是快速傅里叶变换（FFT），这是一种可减少执行时间的DFT计算方法。许多工具箱函数（包括Z域频率响应、频谱和倒频谱分析，以及一些滤波器设计和实现函数）都支持FFT。

Mworks环境提供fft和ifft函数，分别用于计算离散傅里叶变换及其逆变换。对于输入序列x及其变换版本X（围绕单位圆的等间距频率的离散时间傅里叶变换），这两个函数实现以下关系
$$
X(k+1)=\displaystyle\sum_{n=0}^{N-1}x(n+1){W}_{N}^{kn}
$$
和
$$
x(n+1)=\frac{1}{N}\displaystyle\sum_{k=0}^{N-1}X(k+1){W}_{N}^{-kn}
$$
在这些方程中，序列下标从 1 而不是 0 开始，因为采用 MATLAB 向量索引方案，并且
$$
{W}_{N}=e^{-j2\pi /N}
$$
**注意** MATLAB 约定是对 `fft` 函数使用负 *j*。这是工程约定；物理和纯数学通常使用正 *j*。

##### Julia

使用单个输入参量 `x` 的 `fft` 计算输入向量或矩阵的 DFT。如果 `x` 是向量，`fft` 计算向量的 DFT；如果 `x` 是矩形数组，`fft` 计算每个数组列的 DFT。

例如，创建时间向量和信号：

In [1]:
using TyPlot  
using FFTW
# Time vector
t = 0:0.01:10-0.01
# Signal
x = sin.(2π*15*t) + sin.(2π*40*t)

1000-element Vector{Float64}:
  0.0
  1.3968022466674206
  0.0
  1.260073510670101
 -1.1755705045849454
 -1.0000000000000004
  2.220446049250313e-16
 -0.6420395219202074
  1.9021130325903064
  0.2212317420824751
  ⋮
 -0.22123174208244334
 -1.902113032590366
  0.6420395219200211
  1.9584334154387761e-13
  1.0000000000000588
  1.1755705045850835
 -1.2600735106700944
 -4.629630012686903e-14
 -1.3968022466675887

计算信号的 DFT 以及变换后的序列的幅值和相位。通过将小幅值变换值设置为零来减少计算相位时的舍入误差。

In [2]:
# Compute DFT of x
y = fft(x)
tol = 1e-6
y[abs.(y) .< tol] .= 0
# Magnitude
m = abs.(y)
# Phase
theta = angle.(y)

1000-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

要以度为单位绘制幅值和相位，请键入以下命令：

In [3]:
# Frequency vector
f = (0:length(y)-1) * 100 / length(y)
# Plot Magnitude
figure()
subplot(2,1,1)
plot(f, m)
xlabel("Frequency") 
ylabel("Magnitude")
title("Magnitude")
xticks([15, 40, 60, 85])
# Plot Phase
subplot(2,1,2)
plot(f, theta .* 180 / π)
xlabel("Frequency") 
ylabel("Phase (degrees)") 
title("Phase")
xticks([15, 40, 60, 85])

4-element Vector{Int64}:
 15
 40
 60
 85

fft 的第二个参量指定变换的点数 n，表示 DFT 的长度：

In [4]:
# Perform FFT with increased length (n=512)
n = 512
t1 = 0:0.01:5.12-0.01
x1 = sin.(2π*15*t1) + sin.(2π*40*t1)
y1 = fft(x1)
tol = 1e-6
y1[abs.(y1) .< tol] .= 0
m1 = abs.(y1)
theta1 = angle.(y1)
f1 = (0:length(y1)-1) * 100 / length(y1)
# Plot Magnitude
figure()
subplot(2, 1, 1)
plot(f1, m1)
xlabel("Frequency")
ylabel("Magnitude")
title("Magnitude")
xticks([15, 40, 60, 85])
# Plot Phase
subplot(2, 1, 2)
plot(f1, theta1 .* 180 / π)
xlabel("Frequency") 
ylabel("Phase (degrees)") 
title("Phase")
xticks([15, 40, 60, 85])

4-element Vector{Int64}:
 15
 40
 60
 85

在本例中，如果输入序列比 `n` 短，`fft` 会用零填充输入序列，如果输入序列比 `n` 长，则会截断序列。如果未指定 `n`，则默认为输入序列的长度。`fft` 的执行时间取决于其执行的 DFT 的长度 `n`；有关该算法的详细信息，请参阅 `fft` 参考页。

**注意** 得到的 FFT 振幅是 `A*n/2`，其中 `A` 是原始振幅，`n` 是 FFT 点数。仅当 FFT 点的数量大于或等于数据样本的数量时，上述情形才成立。如果 FFT 点数小于数据样本数，则 FFT 振幅比原始振幅低上述量。

离散傅里叶逆变换函数 `ifft` 也接受输入序列以及可选的变换所需点数。尝试以下示例；原始序列 `x` 和重新构造的序列是相同的（在舍入误差内）。

In [5]:
# Generate signal x
t = 0:1/255:1
x = sin.(2π*120*t)
# Compute y as the real part of the inverse FFT of FFT of x
y = real(FFTW.ifft(fft(x)))
# Plot the difference between x and y
figure()
plot(t, x .- y)
xlabel("Time")
ylabel("Difference")
title("Difference between x and y")

PyObject <objects.mw_text.CTitle object at 0x000001A13E5DE448>

该工具箱还包括二维 FFT 及其逆变换的函数，即 fft2 和 ifft2。这些函数对于二维信号或图像处理非常有用。格兹尔函数是计算 DFT 的另一种算法，它也包含在工具箱中。此函数可高效计算长信号中一部分的 DFT。
有时可以方便地重新排列 fft 或 fft2 函数的输出，使零频率分量位于序列的中心。函数 fftshift 将零频率分量移至向量或矩阵的中心。